<a href="https://colab.research.google.com/github/AKookani/NLP/blob/main/Q5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q1: Sentiment Analysis with Naive Bayes Classifier(50 Points)

**Objective:**

You are tasked with implementing a Naive Bayes classifier for sentiment analysis. The provided code is incomplete, and your goal is to complete the missing parts. Additionally, you should train the classifier on a small dataset and analyze its performance.

**Tasks:**

1.**Complete the Code (35 points)**: Fill in the missing parts in the provided Python code for the Naive Bayes classifier. Pay special attention to the `extract_features` function.

2.**Train and Test**: Train the Naive Bayes classifier on the training data and test it on a separate test set. Evaluate the accuracy of the classifier.

3.**Analysis (15 points)**: Discuss the results. Identify any misclassifications and try to understand why the classifier may fail in those cases. Provide examples of sentences that were not predicted correctly and explain possible reasons.


In [1]:
import random
import math
import string
from collections import defaultdict

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import movie_reviews
import nltk

# Download NLTK resources
nltk.download('movie_reviews')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
def get_features(tokens):
    # Remove punctuation
    tokens = [word for word in tokens if word not in string.punctuation]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    # Perform stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    return tokens

In [3]:
class NaiveBayesClassifier:
    def __init__(self, classes):
        """
        Initializes the classifier with necessary data structures.
        :param classes: A set of possible class labels (e.g., {'pos', 'neg'}).
        """
        self.classes = classes  # Stores class labels
        self.class_probs = defaultdict(float)  # Prior probabilities of each class
        self.feature_probs = defaultdict(lambda: defaultdict(float))  # Conditional probabilities of features
        self.vocab = set()  # Set to store unique vocabulary (all unique words)

    def train(self, training_data, smoothing=1.0):
        """
        Trains the Naive Bayes classifier using the training data.
        :param training_data: A list of (tokens, label) tuples where tokens are words from the text.
        :param smoothing: Laplace smoothing factor (default is 1.0).
        """
        # Dictionaries to count class occurrences and feature occurrences per class
        class_counts = defaultdict(int)
        feature_counts = defaultdict(lambda: defaultdict(int))

        # Iterate through the training data to populate counts
        for tokens, label in training_data:
            # Count the occurrences of each class
            class_counts[label] += 1

            # Extract features (cleaned tokens) and count their occurrences for the current class
            features = get_features(tokens)
            for feature in features:
                self.vocab.add(feature)  # Add to the vocabulary
                feature_counts[label][feature] += 1  # Increment count of the feature for the class

        # Total number of training samples
        total_samples = sum(class_counts.values())

        # Compute prior probabilities for each class
        self.class_probs = {label: count / total_samples for label, count in class_counts.items()}

        # Compute conditional probabilities (with Laplace smoothing)
        for label in self.classes:
            # Total number of features (words) in the current class
            total_features = sum(feature_counts[label].values())
            for feature in self.vocab:
                # Apply Laplace smoothing using the smoothing parameter
                self.feature_probs[label][feature] = (
                    (feature_counts[label][feature] + smoothing) / (total_features + smoothing * len(self.vocab))
                )

    def classify(self, features):
        """
        Predicts the class of given features using the Naive Bayes formula.
        :param features: A list of features (tokens) extracted from text.
        :return: The predicted class label.
        """
        # Dictionary to store log probabilities for each class
        log_probs = {}

        # Calculate log probabilities for each class
        for label in self.classes:
            # Start with the log of the prior probability of the class
            log_prob = math.log(self.class_probs[label])
            for feature in features:
                # Add the log of the conditional probability of each feature (if in vocab)
                if feature in self.vocab:
                    log_prob += math.log(self.feature_probs[label][feature])
            log_probs[label] = log_prob  # Store the total log probability for the class

        # Return the class with the highest log probability
        return max(log_probs, key=log_probs.get)

**Parameter Tuning Function**

In [4]:
def tune_laplace_smoothing(training_data, test_data, smoothing_values):
    """
    Tunes the Laplace smoothing parameter by evaluating the classifier's performance
    on different values of the smoothing parameter.
    :param training_data: The training dataset (list of (tokens, label) tuples).
    :param test_data: The testing dataset (list of (tokens, label) tuples).
    :param smoothing_values: A list of smoothing values to test.
    :return: A list of tuples containing smoothing value, training accuracy, and testing accuracy.
    """
    results = []
    for smoothing in smoothing_values:
        # Create a new classifier instance
        classifier = NaiveBayesClassifier(classes)

        # Train the classifier with the current smoothing value
        classifier.train(training_data, smoothing=smoothing)

        # Calculate training and testing accuracy
        train_accuracy = calculate_accuracy(training_data, classifier)
        test_accuracy = calculate_accuracy(test_data, classifier)

        # Append results
        results.append((smoothing, train_accuracy, test_accuracy))
    return results

In [5]:
# Load the movie reviews dataset from NLTK
data = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

random.shuffle(data)

# Shuffle the dataset for randomness
random.shuffle(data)

# Split the dataset into training and testing sets
split_ratio = 0.8
split_index = int(len(data) * split_ratio)
train_set = data[:split_index]
test_set = data[split_index:]

# Train the Naive Bayes classifier
classes = set(sentiment for _, sentiment in train_set)
classifier = NaiveBayesClassifier(classes)
classifier.train(train_set)

def calculate_accuracy(dataset, classifier):
    """
    Evaluates the accuracy of the given classifier on the specified dataset.
    :param dataset: A list of (tokens, label) tuples.
    :param classifier: The Naive Bayes classifier instance.
    :return: The accuracy (fraction of correct predictions).
    """
    correct_predictions = 0  # Counter for correct predictions

    # Iterate over the dataset and classify each example
    for example in dataset:
        tokens, true_sentiment = example  # Extract tokens and true class label
        features = get_features(tokens)  # Extract features from tokens
        predicted_sentiment = classifier.classify(features)  # Predict the class label
        if predicted_sentiment == true_sentiment:  # Check if the prediction is correct
            correct_predictions += 1

    # Calculate and return accuracy
    return correct_predictions / len(dataset)

**Running Parameter Tuning**

In [ ]:
def evaluate_with_seed(seed, train_set, test_set, smoothing_values):
    print(f"\nEvaluating with seed: {seed}")
    random.seed(seed)  # Set seed for reproducibility
    random.shuffle(data)  # Shuffle dataset

    # Split into train and test sets
    split_index = int(len(data) * 0.8)
    train_set = data[:split_index]
    test_set = data[split_index:]

    results = []
    for smoothing in smoothing_values:
        classifier = NaiveBayesClassifier(classes)
        classifier.train(train_set, smoothing=smoothing)
        train_acc = calculate_accuracy(train_set, classifier)
        test_acc = calculate_accuracy(test_set, classifier)
        results.append((smoothing, train_acc, test_acc))

    print("Smoothing | Train Accuracy | Test Accuracy")
    for smoothing, train_acc, test_acc in results:
        print(f"{smoothing:.1f}       | {train_acc:.2f}          | {test_acc:.2f}")

# Test with multiple seeds and smoothing values
smoothing_values = [0.5, 1.0, 2.0]
seeds = [42, 99, 123]  # Example seeds for reproducibility

for seed in seeds:
    evaluate_with_seed(seed, data, data, smoothing_values)


Evaluating with seed: 42
Smoothing | Train Accuracy | Test Accuracy
0.5       | 0.98          | 0.83
1.0       | 0.97          | 0.82
2.0       | 0.96          | 0.83

Evaluating with seed: 99
Smoothing | Train Accuracy | Test Accuracy
0.5       | 0.97          | 0.79
1.0       | 0.97          | 0.81
2.0       | 0.96          | 0.81

Evaluating with seed: 123
Smoothing | Train Accuracy | Test Accuracy
0.5       | 0.97          | 0.81
1.0       | 0.96          | 0.82
2.0       | 0.96          | 0.82


# Q2 : Rule-Based and HMM-Based Tagging ( 50 Points )

### Part-of-Speech Tagging with Rule-Based and Hidden Markov Models
This notebook demonstrates how to implement a basic POS tagger. Students will complete sections to:
1. Build a rule-based POS tagger.
2. Implement a sequence model-based POS tagger using Hidden Markov Models (HMM).
3. Evaluate the performance of both approaches.


#### import libraries

In [6]:
# Import necessary libraries
import nltk
from nltk.corpus import treebank
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split

#### load and explore dataset

In [8]:
# Ensure the dataset is downloaded
nltk.download('treebank')
nltk.download('universal_tagset')

# Load the dataset
sentences = treebank.tagged_sents(tagset='universal')

# Task 1: Print the first 5 tagged sentences to understand the dataset
print(sentences[:5])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


#### preprocessing

In [9]:
# Split the dataset into training and testing sets
train_data, test_data = train_test_split(sentences, test_size=0.2, random_state=42)

# Task 2: Print the number of training and testing sentences
print(f"Training sentences: {len(train_data)}, Testing sentences: {len(test_data)}")

Training sentences: 3131, Testing sentences: 783


#### rule-based pos tagging

In [10]:
# Define a rule-based tagger
# Task 3: Fill in the rules and the default tagger
patterns = [
    (r'.*ing$', 'VERB'),    # Gerunds
    (r'.*ed$', 'VERB'),     # Past tense verbs
    (r'.*es$', 'VERB'),     # 3rd person singular present verbs
    (r'.*ly$', 'ADV'),      # Adverbs
    (r'.*\'s$', 'NOUN'),    # Possessive nouns
    (r'.*s$', 'NOUN'),      # Plural nouns
    (r'^\d+$', 'NUM'),      # Numbers
    (r'.*', 'NOUN')         # Default: nouns
]

# Task 4: Implement the rule-based tagger using nltk.RegexpTagger
rule_based_tagger = nltk.RegexpTagger(patterns)  # Replace 'None' with your solution

#### evaluate rule-based tagger

In [20]:
from nltk.metrics import accuracy
# Task 5

# Extract the words (tokens) from the test data
test_words = [[word for word, tag in sentence] for sentence in test_data] # Get words from each sentence

# Extract the true tags (gold standard tags)
test_tags = [[tag for word, tag in sentence] for sentence in test_data] # Get true tags from each sentence

# Get the predicted tags from the rule-based tagger
predicted_tags = [rule_based_tagger.tag(sentence) for sentence in test_words]

# Calculate the accuracy
accuracy_rule_based = accuracy(
    [tag for sent in test_tags for tag in sent], # Flatten the true tags
    [tag for sent in predicted_tags for _, tag in sent] # Flatten the predicted tags
)
print(f"Accuracy of Rule-Based Tagger: {accuracy_rule_based:.2f}")

Accuracy of Rule-Based Tagger: 0.33


#### HMM-based pos tagging

In [14]:
# Train an HMM-based tagger
# Task 6: Extract tag and word sequences from the training data
def preprocess_hmm_data(tagged_sentences):
    """
    Extracts sequences of words and tags from tagged sentences.
    """
    word_sequences = [[word for word, _ in sent] for sent in tagged_sentences]
    tag_sequences = [[tag for _, tag in sent] for sent in tagged_sentences]
    return word_sequences, tag_sequences

train_words, train_tags = preprocess_hmm_data(train_data)
test_words, test_tags = preprocess_hmm_data(test_data)

# Train transition and emission probabilities
tags = [tag for sentence in train_tags for tag in sentence]
tag_freq = Counter(tags)
tag_bigram_freq = Counter([tuple(sentence[i:i+2]) for sentence in train_tags for i in range(len(sentence)-1)])

# Task 7: Transition probabilities
def train_transition_probs(tag_bigram_freq, tag_freq):
    """
    Compute transition probabilities P(tag2|tag1).
    """
    transition_probs = {}
    for bigram, count in tag_bigram_freq.items():
      tag1, tag2 = bigram
      transition_probs[(tag1, tag2)] = count / tag_freq[tag1] # Probability: count(tag1, tag2) / count(tag1)
    return transition_probs

# Task 8: Implement the emission probability training
def train_emission_probs(tagged_sentences):
    """
    Compute emission probabilities P(word|tag).
    """
    emission_probs = defaultdict(Counter)
    for sentence in tagged_sentences:
        for word, tag in sentence:
            emission_probs[tag][word] += 1 # Count word occurrences for each tag
    for tag in emission_probs:
        total = sum(emission_probs[tag].values()) # Total words tagged as 'tag'
        for word in emission_probs[tag]:
            emission_probs[tag][word] /= total # Normalize to get probabilities
    return emission_probs

# Calculate probabilities
transition_probs = train_transition_probs(tag_bigram_freq, tag_freq)
emission_probs = train_emission_probs(train_data)

#### implement HMM decoding

In [17]:
# Task 9: Implement Viterbi algorithm for decoding
def viterbi_algorithm(sentence, transition_probs, emission_probs, tag_freq):
    """
    Finds the most likely sequence of tags for a given sentence using the Viterbi algorithm.
    """
    states = list(tag_freq.keys()) # All possible tags
    n = len(sentence) # Length of the sentence
    viterbi = np.zeros((len(states), n)) # Matrix to store probabilities
    backpointer = np.zeros((len(states), n), dtype=int) # Matrix to store backpointers

    # Initialization for the first word
    for i, state in enumerate(states):
        viterbi[i, 0] = (
            transition_probs.get(('<s>', state), 1e-6) * # P(tag|start)
            emission_probs[state].get(sentence[0], 1e-6) # P(word|tag)
        )
        backpointer[i, 0] = 0

    # Recursion for the rest of the sentence
    for t in range(1, n):
        for i, state in enumerate(states):
            max_prob = -1
            max_state = 0
            for j, prev_state in enumerate(states):
                prob = (
                    viterbi[j, t - 1] * # Previous probability
                    transition_probs.get((prev_state, state), 1e-6) * # P(current_tag|prev_tag)
                    emission_probs[state].get(sentence[t], 1e-6) # P(word|current_tag)
                )
                if prob > max_prob: # Update max probability and state
                    max_prob = prob
                    max_state = j
            viterbi[i, t] = max_prob
            backpointer[i, t] = max_state

    # Backtrace to find the best tag sequence
    best_path = []
    last_state = np.argmax(viterbi[:, -1]) # Start with the tag having the highest probability
    best_path.append(states[last_state])
    for t in range(n - 1, 0, -1):
        last_state = backpointer[last_state, t]
        best_path.insert(0, states[last_state]) # Insert at the beginning

    return best_path

# Predict tags for a sentence
test_sentence = test_words[0]
predicted_tags = viterbi_algorithm(test_sentence, transition_probs, emission_probs, tag_freq)
print(f"Sentence: {' '.join(test_sentence)}")
print(f"Predicted Tags: {predicted_tags}")

Sentence: For the Agency for International Development , appropriators approved $ 200 million *U* in secondary loan guarantees under an expanded trade credit insurance program , and total loan guarantees for the Overseas Private Investment Corp. are increased *-3 by $ 40 million *U* over fiscal 1989 as part of the same Poland package .
Predicted Tags: ['ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', '.', 'PRON', 'VERB', '.', 'NUM', 'NUM', 'X', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'DET', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN', '.', 'CONJ', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'DET', 'ADJ', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'VERB', 'X', 'ADP', '.', 'NUM', 'NUM', 'X', 'ADP', 'ADJ', 'NUM', 'ADP', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', '.']


#### evaluate HMM-based tagger

In [18]:
# Evaluate the accuracy of the HMM-based tagger
# Task 10: Implement accuracy calculation for HMM-based tagger
correct = 0
total = 0

for sentence, true_tags in zip(test_words, test_tags):
    predicted_tags = viterbi_algorithm(sentence, transition_probs, emission_probs, tag_freq)
    correct += sum(p == t for p, t in zip(predicted_tags, true_tags)) # Count matches
    total += len(true_tags) # Total tags

accuracy_hmm = correct / total
print(f"Accuracy of HMM-Based Tagger: {accuracy_hmm:.2f}")

Accuracy of HMM-Based Tagger: 0.94


In [19]:
for seed in [42, 123, 2024]:
    train_data, test_data = train_test_split(sentences, test_size=0.2, random_state=seed)
    train_words, train_tags = preprocess_hmm_data(train_data)
    test_words, test_tags = preprocess_hmm_data(test_data)
    transition_probs = train_transition_probs(tag_bigram_freq, tag_freq)
    emission_probs = train_emission_probs(train_data)

    correct = 0
    total = 0
    for sentence, true_tags in zip(test_words, test_tags):
        predicted_tags = viterbi_algorithm(sentence, transition_probs, emission_probs, tag_freq)
        correct += sum(p == t for p, t in zip(predicted_tags, true_tags))
        total += len(true_tags)

    accuracy_hmm = correct / total
    print(f"Accuracy of HMM-Based Tagger with seed {seed}: {accuracy_hmm:.2f}")

Accuracy of HMM-Based Tagger with seed 42: 0.94
Accuracy of HMM-Based Tagger with seed 123: 0.93
Accuracy of HMM-Based Tagger with seed 2024: 0.93


# Submission Instructions:

1.Submit a Google Colab notebook containing your completed code and experimentation results.

2.Include comments and explanations in your code to help understand the implemented logic.

3.Clearly present the results of your parameter tuning in the notebook.

4.Provide a brief summary of your findings and insights in the conclusion section.

**Additional Notes:**
*   Ensure that the notebook runs successfully in Google Colab.
*   Experiment with various seed texts to showcase the diversity of generated text.
*   Document any issues encountered during experimentation and how you addressed them.

**Grading:**
*   Each task will be graded out of the specified points.
*   Points will be awarded for correctness, clarity of code, thorough experimentation, and insightful analysis.